In [0]:
# %tensorflow_version 1.x
import os
from datetime import datetime, timedelta
from pytz import timezone    
import numpy as np
import tensorflow as tf
import cv2
import time
import pandas as pd
from google.colab.patches import cv2_imshow
import math

import imutils

india = timezone('Asia/Kolkata')

In [3]:
input_video_path = "Mahathi_dribble.mp4"
output_video_path = "mahathi_detect.avi"

cap = cv2.VideoCapture(input_video_path)
writer = None
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"DIVX")
writer = cv2.VideoWriter(output_video_path, fourcc, 30, (frame_width, frame_height), True)

frame_count = 0
while True:
  ret, img = cap.read()
  cv2_imshow(img)
  if ret is None:
    break
  writer.write(img)
  frame_count += 1
  print(frame_count)

cap.release()
writer.release()

AttributeError: ignored

In [0]:
bodyParts = ["nose", "leftEye", "rightEye", "leftEar", "rightEar", "leftShoulder", "rightShoulder", "leftElbow", "rightElbow", "leftWrist", "rightWrist", "leftHip", "rightHip", "leftKnee", "rightKnee", "leftAnkle", "rightAnkle", "neck"]
bodyLines = [
  ("leftShoulder", "rightShoulder"),
  ("leftShoulder", "leftElbow"),
  ("rightShoulder", "rightElbow"), 
  ("leftElbow", "leftWrist"),
  ("rightElbow", "rightWrist"),
  ("leftShoulder", "leftHip"),
  ("rightShoulder", "rightHip"),
  ("leftHip", "rightHip"),
  ("leftHip", "leftKnee"),
  ("rightHip", "rightKnee"),
  ("leftKnee", "leftAnkle"),
  ("rightKnee", "rightAnkle"),
  ("nose", "neck")
]

In [0]:
def sigmoid (x):
  return (1/(1+math.exp(-x)))

In [0]:
def midpoint (pt1, pt2):
  x1, y1 = pt1
  x2, y2 = pt2
  mx = int((x1+x2) / 2)
  my = int((y1+y2) / 2)

  return (mx, my)

In [0]:
def createKeyPointDict (xCoords, yCoords):
  coordinates_dict = dict()
  for idx,bodyPart in enumerate(bodyParts):
    if (bodyPart == "neck"):
      coordinates_dict[bodyPart] = midpoint(coordinates_dict["leftShoulder"], coordinates_dict["rightShoulder"])
    else:
      coordinates_dict[bodyPart] = (xCoords[idx], yCoords[idx])

  return coordinates_dict

In [6]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="multi_posenet_mv1_075.tflite")
interpreter.allocate_tensors()

# Get input and output tensor details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_height = input_details[0]['shape'][1]
input_width = input_details[0]['shape'][2]

print("Shape of image accepted by Posenet model", input_height, input_width)

Shape of image accepted by Posenet model 337 337


In [0]:
def postureDetector(image, H, W, writer):
  # image = cv2.imread("sitting_man_3.jpeg")

  image = cv2.resize(image, (input_width, input_height))

  input_data = np.expand_dims(image, axis=0)
  input_data = (np.float32(input_data) - 127.5) / 127.5
  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()

  heatmap = interpreter.get_tensor(output_details[0]['index'])
  heatmap_results = np.squeeze(heatmap)

  offset = interpreter.get_tensor(output_details[1]['index'])
  offset_results = np.squeeze(offset)

  # print("Shape of heatmap: ", heatmap_results.shape)

  # print("Shape of offset", offset_results.shape)

  # Find the output stride
  output_stride = int((input_width-1) / (heatmap_results.shape[0]-1))

  # Calculate the c and y positions of the keypoints from the heatmap

  height = len(heatmap_results)
  width = len(heatmap_results[0])
  numKeypoints = len(heatmap_results[0][0])

  keypointPositions = []
  for keypoint in range(0, numKeypoints):
    maxVal = heatmap_results[0][0][keypoint]
    maxRow = 0
    maxCol = 0

    for row in range(0, height):
      for col in range(0, width):
        if heatmap_results[row][col][keypoint] > maxVal:
          maxVal = heatmap_results[row][col][keypoint]
          maxRow = row
          maxCol = col

    keypointPositions.append(tuple((maxRow, maxCol)))

  # Calculate x and y coordinates of the keypoints

  xCoords = []
  yCoords = []
  confidenceScores = []

  for idx,value in enumerate(keypointPositions):
    yCoords.append(int(value[0] * output_stride + offset_results[value[0]][value[1]][idx]))
    xCoords.append(int(value[1] * output_stride + offset_results[value[0]][value[1]][idx+numKeypoints]))

    confidenceScores.append(sigmoid(heatmap_results[value[0]][value[1]][idx]))

  coordinates_dict = createKeyPointDict(xCoords, yCoords)
  # print(coordinates_dict)

  for line_end_points in bodyLines:
    p1 = coordinates_dict[line_end_points[0]]
    p2 = coordinates_dict[line_end_points[1]]
    cv2.line(image, p1, p2, (0, 255, 0), 1)

  for value in coordinates_dict.values():
    cv2.circle(image, value, radius = 2, color = (0, 0, 255), thickness = -1)

  output_image = cv2.resize(image, (W, H))

  # cv2_imshow(output_image)

  writer.write(output_image)

In [12]:
input_video_path = "mahathi_dribble.mp4"
output_video_path = "mahathi_posture.avi"

cap = cv2.VideoCapture(input_video_path)
writer = None
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"DIVX")
if (frame_height > frame_width):
  writer = cv2.VideoWriter(output_video_path, fourcc, 30, (frame_height, frame_width), True)
else:
  writer = cv2.VideoWriter(output_video_path, fourcc, 30, (frame_width, frame_height), True)

while True:
  ret, img = cap.read()
  if ret is None or img is None:
    break

  img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)

  (H, W) = img.shape[:2]
  print("Actual Image shape", H, W)
  postureDetector(img, H, W, writer)

# ret, img = cap.read()
# img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
# cv2.imwrite("mahathi_dribble.jpg", img)
# image = cv2.imread("mahathi_dribble.jpg")
# (H, W) = img.shape[:2]
# print("Actual Image shape", H, W)
# postureDetector(img, H, W, writer)

cap.release()
writer.release()

Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1280 720
Actual Image shape 1

In [13]:
!ffmpeg -i mahathi_posture.avi mahathi_posture.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [14]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('mahathi_posture.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)